# Elasticity
<strong>J. M. Maas @ IfB, ETH Zürich, v. 2026-01-19</strong>

<div class='alert alert-info'>
    Select ▸▸ <strong>Restart the kernel and run all cells</strong> to run the widgets in this notebook and explore the elasticity data.
</div>

In [ ]:
# Required libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
%matplotlib widget
from datetime import datetime

# Import required modules
import utils.scleronomic_utils as scutils

The next cell imports the data points of the diagram.

In [ ]:
# Import elasticity data
df = pd.read_csv('data/01_elasticity/elasticity_data.csv', index_col=['component', 'sample'])
display(df)

The next cell creates a widget to explore the elasticity data.

In [ ]:
# Create widget
df_export = [pd.DataFrame()]

# Fit moisture-dependent function over constant distribution
def fit_moisture_dependence(data, engineering_constant, order=3, label='(not specified)'):
    '''
    Fits the moisture dependence over a data distribution.
    Input:
        data                 (pd.DataFrame)   : Data points that are fitted. Requires colums 'constant'
                                                with the data values, 'rh' with the relative humidity,
                                                and 'mc' with the moisture content.
        engineering_constant (str)            : Engineering constant name. Determines the starting
                                                parameters and settings of the fitting algorithm.
    Output:
        fitting_fun          (function)       : Fitting function in the structure
                                                fitting_fun = lambda mc, p: ... with fitted parameter
                                                array p and moisture content mc.
        df_params            (pd.DataFrame)   : Fitted parameters and R2 value of the fitting.
        model                (lf.ModelResult) : Lmfit model result class.
    '''
    # Fitting constraints
    E0 = data['constant'].mean()
    if 'nu' in engineering_constant:  # select parameters based on type
        p0 = p0 = (order*[-0.001] + [E0])[-order-1:]
        bounds = (order*[(-1, 1)] + [(0, 10*E0)])[-order-1:]
    elif engineering_constant in ['E_RL', 'E_LR', 'E_TL', 'E_LT']:  # lateral strains with very large stiffness
        p0 = (order*[-1] + [E0])[-order-1:]
        bounds = (order*[(-1000, 1000)] + [(0.1*E0, 10*E0)])[-order-1:]
    else:  # default
        p0 = (order*[1e-8] + [E0])[-order-1:]
        bounds = (order*[(-1, 1)] + [(-10, 10), (-10, 10), (0.1*E0, 10*E0)])[-order-1:]

    # Least-squares fitting quadratic function
    fitting_fun = lambda mc, p: p[0]*mc**2 + p[1]
    sol = scutils.lsq_fitting(f=fitting_fun, xdata=data['mc'].to_numpy(), ydata=data['constant'].to_numpy(), p0=p0[::2], method='lmfit', options=dict(maxiter=2000, bounds=bounds[::2]))
    p, success, R2 = sol['x'], sol['success'], sol['R2']
    model = sol['obj']
    if not sol['success']:
        print(f'Warning: No fitting success for {label}: {sol.message}')
    
    # Return results
    df_data = [(f'p{i:d}', p_i) for i, p_i in enumerate(p)] + [(f'Std_p{i:d}', err_i) for i, err_i in enumerate(sol['err'])] + [('R2', R2)]
    df_params = pd.DataFrame(data=dict(df_data), index=pd.Index(data=[engineering_constant], name='constant'))
    return fitting_fun, df_params, model


# Visualization function
def visualize_moisture(df, engineering_constant, violin_plot=False, fit_plot=True, print_formular=True, drop_outliers=True, fit_average=False, show_confidence=True, normalize=False, sigma=1, ax=None):
    # Select data
    df_constants = df.loc[engineering_constant].copy()
    if drop_outliers:
        df_constants.query('not outlier', inplace=True)
    df_constants.dropna(inplace=True)
    
    # Initialize plot
    if ax is None:
        fig, ax = plt.subplots()
    ax.cla()
    print()
    
    # Plot settings
    scatter_plot = not violin_plot
    colors = ['cornflowerblue', 'chocolate', '#454545']
    fcs = [(1.0, 1.0, 1.0, 0.0), (1.0, 1.0, 1.0, 0.0), (1.0, 1.0, 1.0, 0.0)]
    ls = ['-', '-', '-.']
    markers = ['o', '^', '']
    formular_position = ([(0.02, 0.12), (0.02, 0.05), (0.02, 0.19)])
    
    # Create violinplot
    if violin_plot:
        sns.violinplot(ax=ax, data=df_constants, x='mc_mean', y='constant', hue='label', palette=colors, native_scale=True, split=True, inner='quart', width=1.2)
    
    # Iterate over unique selected types
    labels, dfs, models = [], [], dict()
    unique_types = df_constants['label'].unique()
    if fit_average: unique_types = list(unique_types) + ['avg']
    for i, index in enumerate(unique_types):
        # Select data
        data = df_constants.copy() if 'avg' in index else df_constants.loc[df_constants['label'] == index].copy()
        data['mc'] *= 100  # calculate with moisture content in [%]
        data.loc[data['mc'].isna(), 'mc'] = data.loc[data['mc'].isna(), 'mc_mean']  # set missing moisture contents to average moisture content

        # Format labels
        label = data.iloc[0]['label']
        if 'avg' in index:
            index = scutils.replace_multiple(unique_types[0], ['c,', 't,'], 2*['avg,'])
            index = index.replace('G_{', 'G_{avg') if 'G_' in index else index
            label = scutils.replace_multiple(label, ['c,', 't,'], 2*['\mathrm{avg},'])
            label = label.replace('G_{', 'G_{\mathrm{avg},') if 'G_' in label else label
        labels.append( label )
    
        # Determine fit
        if fit_plot:
            # Apply fitting
            fitting_fun, df_params, model = fit_moisture_dependence(data, engineering_constant, label=labels[i], order=2)
            index_clean = index.strip('{}$').replace('{', '').replace(',', '')  # remove Latex notation
            df_params.index = pd.Index( data=[index_clean], name='constant' )
            p = df_params.iloc[0][[c for c in df_params.columns if ('p' in c and not 'std' in c.lower())]].to_numpy()
            R2 = df_params.iloc[0]['R2']
            dfs.append(df_params)
            models[index_clean] = model
        
            # Plot fit
            fit_mc = np.linspace(0, 25, 100)
            fit_constant = fitting_fun(fit_mc, p)
            normalize_factor = 1.0 / fit_constant[0] if normalize else 1.0
            fit_constant *= normalize_factor
            ax.plot(fit_mc, fit_constant, zorder=4, c=colors[i], ls=ls[i])

            # Plot error
            if show_confidence and not ('avg' in index):
                fit_confidence = model.eval_uncertainty(x=fit_mc, sigma=sigma) * normalize_factor
                fit_prediction = model.dely_predicted * normalize_factor
                ax.fill_between(fit_mc, fit_constant-fit_confidence, fit_constant+fit_confidence, zorder=2, color=colors[i], ec='none', alpha=0.5)
                ax.fill_between(fit_mc, fit_constant-fit_prediction, fit_constant+fit_prediction, zorder=1, color=colors[i], ec='none', alpha=0.3)
    
            # Print formula
            formular_string = labels[i].replace('$', '') + r'(\omega) = '
            exponent = [2, 0]  # exponents in the written equation
            for j in range(len(p)):
                sign = '+' if np.sign(p[j]) >= 0 else '-'
                P = scutils.round_to_significant_digits(np.round(np.abs(p[j]), 4), 3)
                if j == 0: formular_string += f'{sign if sign == "-" else ""}{P}'
                elif j == 1: formular_string += f'{sign} {P}'
                else: formular_string += rf'{sign} {P}'
                if exponent[j] > 1: formular_string += rf' \omega^{{ {exponent[j]} }}'
                elif exponent[j] > 0: formular_string += rf' \omega'
            formular_string = rf'${formular_string}$'
            if print_formular:
                text = ax.text(formular_position[i][0], formular_position[i][1], formular_string, c=colors[i], ha='left', va='baseline', transform=ax.transAxes, size='medium')
        else:
            normalize_factor = 1.0 / data.query('mc < 10.0')['constant'].mean() if normalize else 1.0
        
        # Plot experimental data
        if scatter_plot:
            ax.scatter(data['mc'], data['constant']*normalize_factor, zorder=3, marker=markers[i], color=colors[i], fc=fcs[i])

    # Assemble data
    df_params = pd.concat(dfs) if len(dfs) > 0 else pd.DataFrame(columns=['R2'], index=pd.Index(data=[], name='constant'))
    R2s = df_params['R2'].to_list()
    
    
    # Create legend
    legend_labels = [label + rf' ($R^2 = {R2:.2f}$)' for label, R2 in zip(labels, R2s)] if len(R2s) > 0 else labels
    legend_symbols = [plt.Line2D([0], [0], color=colors[i], lw=2, linestyle='-', marker=markers[i], mfc=fcs[i], mec=colors[i]) for i in range(len(legend_labels))]
    legend = ax.legend(legend_symbols, legend_labels, loc='upper right')
    legend.set_zorder(10)
    
    # String manipulation function
    def replace_multiple(s, old_values, new_values):
        for old_value, new_value in zip(old_values, new_values):
            s = s.replace(old_value, new_value)
        return s
    
    # Format axes
    ax.set_xlim([0, 25])
    ax.grid()
    ax.set_xlabel('Moisture content [%]')
    ax.set_ylabel(' / '.join(labels) + ' [MPa]')

    # Return results
    df_points = df_constants
    df_export[0] = df_points
    return df_params, models, df_points


# Export function
def export_csv(button):
    if len(df_export) > 0:
        export_path = f'export_elasticity_{w_component.value}.csv'
        df_export[0].to_csv(export_path)
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with w.children[-1]:
            w.children[-1].clear_output()
            print(f'({timestamp}) Successfully saved data to {export_path}.')
button = widgets.Button(description='Export CSV', button_style='primary', icon='save')
button.on_click(export_csv)
display(button)



# Initialize figure
fig, ax = plt.subplots()

# Add interactivity
w_component = widgets.Dropdown(options=['E_RR', 'E_TT', 'E_LL', 'E_RT', 'E_RL', 'E_TL', 'G_RT', 'G_RL', 'G_TL', 'nu_RT', 'nu_LT', 'nu_RL', 'nu_TL', 'nu_TR', 'nu_LR'], description='component')
w = widgets.interactive(
    visualize_moisture,
    df=widgets.fixed(df),
    engineering_constant=w_component,
    sigma=widgets.FloatSlider(value=1, min=0, max=3),
    ax=widgets.fixed(ax)
)
display(w)

Use the control elements to explore the data.